In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e12/sample_submission.csv
/kaggle/input/playground-series-s4e12/train.csv
/kaggle/input/playground-series-s4e12/test.csv


In [2]:
train_df = pd.read_csv('/kaggle/input/playground-series-s4e12/train.csv',low_memory=False,parse_dates=['Policy Start Date'])
test_df = pd.read_csv('/kaggle/input/playground-series-s4e12/test.csv',low_memory=False, parse_dates=['Policy Start Date'])
train_df

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,...,NaN,5.0,372.0,3.0,2023-05-03 15:21:39.257696,Poor,No,Daily,Apartment,1303.0
1199996,1199996,54.0,Male,35786.0,Divorced,NaN,Master's,Self-Employed,11.483482,Rural,...,NaN,10.0,597.0,4.0,2022-09-10 15:21:39.134960,Poor,No,Weekly,Apartment,821.0
1199997,1199997,19.0,Male,51884.0,Divorced,0.0,Master's,NaN,14.724469,Suburban,...,0.0,19.0,NaN,6.0,2021-05-25 15:21:39.106582,Good,No,Monthly,Condo,371.0
1199998,1199998,55.0,Male,NaN,Single,1.0,PhD,NaN,18.547381,Suburban,...,1.0,7.0,407.0,4.0,2021-09-19 15:21:39.190215,Poor,No,Daily,Apartment,596.0


In [3]:
train_df.isna().sum()

id                           0
Age                      18705
Gender                       0
Annual Income            44949
Marital Status           18529
Number of Dependents    109672
Education Level              0
Occupation              358075
Health Score             74076
Location                     0
Policy Type                  0
Previous Claims         364029
Vehicle Age                  6
Credit Score            137882
Insurance Duration           1
Policy Start Date            0
Customer Feedback        77824
Smoking Status               0
Exercise Frequency           0
Property Type                0
Premium Amount               0
dtype: int64

In [4]:
#Copy new data to work on for both train and test data
work_train_df = train_df.copy()



#Check out Column with date format
work_train_df['Policy Start Date']

0         2023-12-23 15:21:39.134960
1         2023-06-12 15:21:39.111551
2         2023-09-30 15:21:39.221386
3         2024-06-12 15:21:39.226954
4         2021-12-01 15:21:39.252145
                     ...            
1199995   2023-05-03 15:21:39.257696
1199996   2022-09-10 15:21:39.134960
1199997   2021-05-25 15:21:39.106582
1199998   2021-09-19 15:21:39.190215
1199999   2020-08-26 15:21:39.155231
Name: Policy Start Date, Length: 1200000, dtype: datetime64[ns]

In [5]:
import matplotlib.pyplot as plt
#Create different columns from Date column
def policy_date_change(dataframe):
    dataframe['Policy Start Date'] = pd.to_datetime(dataframe['Policy Start Date'])
    dataframe['year'] = dataframe['Policy Start Date'].dt.year
    dataframe['Month'] = dataframe['Policy Start Date'].dt.month
    dataframe['dayofweek'] = dataframe['Policy Start Date'].dt.dayofweek
    dataframe['saleday'] = dataframe['Policy Start Date'].dt.day
    dataframe['dayofyear'] = dataframe['Policy Start Date'].dt.dayofyear
    dataframe.drop('Policy Start Date', axis=1, inplace=True)
    return dataframe
#Remove Policy date column


policy_date_change(dataframe=work_train_df)


/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount,year,Month,dayofweek,saleday,dayofyear
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,Poor,No,Weekly,House,2869.0,2023,12,5,23,357
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,Average,Yes,Monthly,House,1483.0,2023,6,0,12,163
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,Good,Yes,Weekly,House,567.0,2023,9,5,30,273
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,Poor,Yes,Daily,Apartment,765.0,2024,6,2,12,164
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,Poor,Yes,Weekly,House,2022.0,2021,12,2,1,335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,...,Poor,No,Daily,Apartment,1303.0,2023,5,2,3,123
1199996,1199996,54.0,Male,35786.0,Divorced,NaN,Master's,Self-Employed,11.483482,Rural,...,Poor,No,Weekly,Apartment,821.0,2022,9,5,10,253
1199997,1199997,19.0,Male,51884.0,Divorced,0.0,Master's,NaN,14.724469,Suburban,...,Good,No,Monthly,Condo,371.0,2021,5,1,25,145
1199998,1199998,55.0,Male,NaN,Single,1.0,PhD,NaN,18.547381,Suburban,...,Poor,No,Daily,Apartment,596.0,2021,9,6,19,262


In [6]:
def new_column(dataframe):
    new_column = dataframe['Annual Income'] / dataframe['Number of Dependents']
    dataframe['Income per dependent']= new_column.round(2)
    return dataframe

In [7]:
new_column(dataframe=work_train_df)

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Smoking Status,Exercise Frequency,Property Type,Premium Amount,year,Month,dayofweek,saleday,dayofyear,Income per dependent
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,No,Weekly,House,2869.0,2023,12,5,23,357,10049.00
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,Yes,Monthly,House,1483.0,2023,6,0,12,163,10559.33
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,Yes,Weekly,House,567.0,2023,9,5,30,273,8534.00
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,Yes,Daily,Apartment,765.0,2024,6,2,12,164,70927.50
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,Yes,Weekly,House,2022.0,2021,12,2,1,335,39651.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,...,No,Daily,Apartment,1303.0,2023,5,2,3,123,inf
1199996,1199996,54.0,Male,35786.0,Divorced,NaN,Master's,Self-Employed,11.483482,Rural,...,No,Weekly,Apartment,821.0,2022,9,5,10,253,NaN
1199997,1199997,19.0,Male,51884.0,Divorced,0.0,Master's,NaN,14.724469,Suburban,...,No,Monthly,Condo,371.0,2021,5,1,25,145,inf
1199998,1199998,55.0,Male,NaN,Single,1.0,PhD,NaN,18.547381,Suburban,...,No,Daily,Apartment,596.0,2021,9,6,19,262,NaN


In [8]:
work_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 26 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1200000 non-null  int64  
 1   Age                   1181295 non-null  float64
 2   Gender                1200000 non-null  object 
 3   Annual Income         1155051 non-null  float64
 4   Marital Status        1181471 non-null  object 
 5   Number of Dependents  1090328 non-null  float64
 6   Education Level       1200000 non-null  object 
 7   Occupation            841925 non-null   object 
 8   Health Score          1125924 non-null  float64
 9   Location              1200000 non-null  object 
 10  Policy Type           1200000 non-null  object 
 11  Previous Claims       835971 non-null   float64
 12  Vehicle Age           1199994 non-null  float64
 13  Credit Score          1062118 non-null  float64
 14  Insurance Duration    1199999 non-

In [9]:
#Cahnge string format to category
def string_to_cat(dataframe):
    for label,content in dataframe.items():
        if pd.api.types.is_string_dtype(content):
            dataframe[label] = content.astype('category')
    return dataframe
    
    

In [10]:
def obj_to_cat(dataframe):
    for label,content in dataframe.items():
        if pd.api.types.is_object_dtype(content):
            dataframe[label] = content.astype('category')
    return dataframe

In [11]:
work_train_df = string_to_cat(dataframe=work_train_df)
work_train_df = obj_to_cat(dataframe=work_train_df)
work_train_df.isna().sum()

id                           0
Age                      18705
Gender                       0
Annual Income            44949
Marital Status           18529
Number of Dependents    109672
Education Level              0
Occupation              358075
Health Score             74076
Location                     0
Policy Type                  0
Previous Claims         364029
Vehicle Age                  6
Credit Score            137882
Insurance Duration           1
Customer Feedback        77824
Smoking Status               0
Exercise Frequency           0
Property Type                0
Premium Amount               0
year                         0
Month                        0
dayofweek                    0
saleday                      0
dayofyear                    0
Income per dependent    150613
dtype: int64

In [12]:
work_train_df.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Smoking Status,Exercise Frequency,Property Type,Premium Amount,year,Month,dayofweek,saleday,dayofyear,Income per dependent
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,No,Weekly,House,2869.0,2023,12,5,23,357,10049.00
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,Yes,Monthly,House,1483.0,2023,6,0,12,163,10559.33
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,Yes,Weekly,House,567.0,2023,9,5,30,273,8534.00
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,Yes,Daily,Apartment,765.0,2024,6,2,12,164,70927.50
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,Yes,Weekly,House,2022.0,2021,12,2,1,335,39651.00


In [13]:
work_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 26 columns):
 #   Column                Non-Null Count    Dtype   
---  ------                --------------    -----   
 0   id                    1200000 non-null  int64   
 1   Age                   1181295 non-null  float64 
 2   Gender                1200000 non-null  category
 3   Annual Income         1155051 non-null  float64 
 4   Marital Status        1181471 non-null  category
 5   Number of Dependents  1090328 non-null  float64 
 6   Education Level       1200000 non-null  category
 7   Occupation            841925 non-null   category
 8   Health Score          1125924 non-null  float64 
 9   Location              1200000 non-null  category
 10  Policy Type           1200000 non-null  category
 11  Previous Claims       835971 non-null   float64 
 12  Vehicle Age           1199994 non-null  float64 
 13  Credit Score          1062118 non-null  float64 
 14  Insurance Duration

In [14]:
def float_to_numeric(dataframe):
    for label,content in dataframe.items():
       if pd.api.types.is_float_dtype(content):
           dataframe[label] = content.astype('float16')
    return dataframe



work_train_df = float_to_numeric(dataframe=work_train_df)

/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)


In [15]:
work_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 26 columns):
 #   Column                Non-Null Count    Dtype   
---  ------                --------------    -----   
 0   id                    1200000 non-null  int64   
 1   Age                   1181295 non-null  float16 
 2   Gender                1200000 non-null  category
 3   Annual Income         1155051 non-null  float16 
 4   Marital Status        1181471 non-null  category
 5   Number of Dependents  1090328 non-null  float16 
 6   Education Level       1200000 non-null  category
 7   Occupation            841925 non-null   category
 8   Health Score          1125924 non-null  float16 
 9   Location              1200000 non-null  category
 10  Policy Type           1200000 non-null  category
 11  Previous Claims       835971 non-null   float16 
 12  Vehicle Age           1199994 non-null  float16 
 13  Credit Score          1062118 non-null  float16 
 14  Insurance Duration

In [16]:
raw_data = work_train_df.copy()
work_train_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 26 columns):
 #   Column                Non-Null Count    Dtype   
---  ------                --------------    -----   
 0   id                    1200000 non-null  int64   
 1   Age                   1181295 non-null  float16 
 2   Gender                1200000 non-null  category
 3   Annual Income         1155051 non-null  float16 
 4   Marital Status        1181471 non-null  category
 5   Number of Dependents  1090328 non-null  float16 
 6   Education Level       1200000 non-null  category
 7   Occupation            841925 non-null   category
 8   Health Score          1125924 non-null  float16 
 9   Location              1200000 non-null  category
 10  Policy Type           1200000 non-null  category
 11  Previous Claims       835971 non-null   float16 
 12  Vehicle Age           1199994 non-null  float16 
 13  Credit Score          1062118 non-null  float16 
 14  Insurance Duration

In [17]:
def replace_inf(dataframe):
    for label, content in dataframe.items():
        if pd.api.types.is_float_dtype(content):
            dataframe[label] = content.replace([np.inf, -np.inf], np.nan)
    return dataframe

In [18]:
work_train_df = replace_inf(dataframe=work_train_df)
work_train_df


/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()


,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Smoking Status,Exercise Frequency,Property Type,Premium Amount,year,Month,dayofweek,saleday,dayofyear,Income per dependent
0,0,19.0,Female,10048.0,Married,1.0,Bachelor's,Self-Employed,22.593750,Urban,...,No,Weekly,House,2868.0,2023,12,5,23,357,10048.0
1,1,39.0,Female,31680.0,Divorced,3.0,Master's,NaN,15.570312,Rural,...,Yes,Monthly,House,1483.0,2023,6,0,12,163,10560.0
2,2,23.0,Male,25600.0,Divorced,3.0,High School,Self-Employed,47.187500,Suburban,...,Yes,Weekly,House,567.0,2023,9,5,30,273,8536.0
3,3,21.0,Male,NaN,Married,2.0,Bachelor's,NaN,10.937500,Rural,...,Yes,Daily,Apartment,765.0,2024,6,2,12,164,NaN
4,4,21.0,Male,39648.0,Single,1.0,Bachelor's,Self-Employed,20.375000,Rural,...,Yes,Weekly,House,2022.0,2021,12,2,1,335,39648.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,1199995,36.0,Female,27312.0,Married,0.0,Master's,Unemployed,13.773438,Urban,...,No,Daily,Apartment,1303.0,2023,5,2,3,123,NaN
1199996,1199996,54.0,Male,35776.0,Divorced,NaN,Master's,Self-Employed,11.484375,Rural,...,No,Weekly,Apartment,821.0,2022,9,5,10,253,NaN
1199997,1199997,19.0,Male,51872.0,Divorced,0.0,Master's,NaN,14.726562,Suburban,...,No,Monthly,Condo,371.0,2021,5,1,25,145,NaN
1199998,1199998,55.0,Male,NaN,Single,1.0,PhD,NaN,18.546875,Suburban,...,No,Daily,Apartment,596.0,2021,9,6,19,262,NaN


In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

# Split data
X = work_train_df.drop('Premium Amount', axis=1)
y = raw_data['Premium Amount']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert all categorical columns to string to avoid mixed types
category_columns = X.select_dtypes(include=['category']).columns
for col in category_columns:
    X[col] = X[col].astype(str)

# Identifying numeric and categorical columns
numeric_columns = X.select_dtypes(include=['int32', 'int64', 'float16', 'float64']).columns
category_columns = X.select_dtypes(include=['category']).columns

# Numerical transformer (impute missing values and scale)
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', StandardScaler())
])

# Ordinal columns (customize as needed)
ordinal_columns = ['Education Level', 'Occupation', 'Location']

# Nominal columns (everything not ordinal)
nominal_columns = list(set(category_columns) - set(ordinal_columns))

# Ordinal encoding transformer
ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('ordinal', OrdinalEncoder())
])

# Nominal encoding transformer (OneHotEncoding)
nominal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),  # 'Unknown' for categorical data
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine all transformers into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numeric_columns),
        ('ordinal', ordinal_transformer, ordinal_columns),
        ('nominal', nominal_transformer, nominal_columns)
    ]
)

# Create a list of models to compare
#models = [
   # ('Linear Regression', LinearRegression()),
   # ('Random Forest', RandomForestRegressor(random_state=42)),
    #('XGBoost', XGBRegressor(random_state=42)),
    #('CatBoost', CatBoostRegressor(random_state=42, verbose=0))  # verbose=0 to suppress output
#]

    # Create a pipeline for each model
#pipeline = Pipeline(steps=[
        #('preprocessor', preprocessor),  # Apply the preprocessing steps
       # ('models', models)  # Add the model
#])
model = RandomForestRegressor()




#Create a votingRegression to combine models

#model = LinearRegression()
    #('linreg',linreg)])

# Define the pipeline
pipeline = Pipeline(steps=[
('preprocessor', preprocessor),
('models', model)
])
pipeline.fit(X_train, y_train)
    
#pipeline.score(X_val_preprocessed, y_val)
score = pipeline.score(X_val, y_val)

#Evaluate individual models
score = pipeline.score(X_val, y_val)
print(f' Linear Regression accuracy score : {score}')


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:522: RuntimeWarning: overflow encountered in cast
  scl = avg_as_array.dtype.type(a.size/avg_as_array.size)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:929: RuntimeWarning: overflow encountered in square
  weight * (y_true - np.average(y_true, axis=0, weights=sample_weight)) ** 2


 Linear Regression accuracy score : 1.0


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:522: RuntimeWarning: overflow encountered in cast
  scl = avg_as_array.dtype.type(a.size/avg_as_array.size)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:929: RuntimeWarning: overflow encountered in square
  weight * (y_true - np.average(y_true, axis=0, weights=sample_weight)) ** 2


In [20]:
policy_date_change(dataframe=test_df)
new_column(dataframe=test_df)
test_df = string_to_cat(dataframe=test_df)
test_df = obj_to_cat(dataframe=test_df)
test_df = float_to_numeric(dataframe=test_df)
test_df = replace_inf(dataframe=test_df)


/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)


In [21]:
#test_df
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['id', 'Age', 'Annual Income', 'Number of Dependents', 'Health Score',
       'Previous Claims', 'Vehicle Age', 'Credit Score', 'Insurance Duration',
       'year', 'Month', 'dayofweek', 'saleday', 'day...
                                                 ('ordinal',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='Unknown',
                                                                                 strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder())]),
                                                  ['Education Level',
                                                   'Occupation', 'Location']),
                                                 ('nominal',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='Unknown',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [])])),
                ('models', RandomForestRegressor())])

In [22]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 25 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   id                    800000 non-null  int64   
 1   Age                   787511 non-null  float16 
 2   Gender                800000 non-null  category
 3   Annual Income         658261 non-null  float16 
 4   Marital Status        787664 non-null  category
 5   Number of Dependents  726870 non-null  float16 
 6   Education Level       800000 non-null  category
 7   Occupation            560875 non-null  category
 8   Health Score          750551 non-null  float16 
 9   Location              800000 non-null  category
 10  Policy Type           800000 non-null  category
 11  Previous Claims       557198 non-null  float16 
 12  Vehicle Age           799997 non-null  float16 
 13  Credit Score          708549 non-null  float16 
 14  Insurance Duration    799998 non-nul

In [23]:
work_test_df = test_df.copy()
#y_preds = pipeline.predict(work_test_df)

In [24]:
val_preds = pipeline.predict(X_val)

In [25]:
from sklearn.metrics import mean_squared_log_error

#mean_squared_log_error(y_val, y_preds)
np.sqrt(mean_squared_log_error(y_val, val_preds))

1.163728156068479

In [26]:
y_preds = pipeline.predict(work_test_df)

In [27]:
submission_df = pd.DataFrame()
submission_df['Id'] = test_df['id']
submission_df['Premium Amount'] = y_preds
submission_df.shape

(800000, 2)

In [28]:
submission_df.to_csv('submission.csv', index=False)